In [94]:
import torch
from torch import nn
import numpy as np
import pandas as pd
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
from tqdm import tqdm

In [121]:
class Config():
    input_paths =  ['2023-wimbledon-1301-player1.csv',
                    '2023-wimbledon-1301-player2.csv',
                    '2023-wimbledon-1302-player1.csv',
                    '2023-wimbledon-1302-player2.csv',
                    '2023-wimbledon-1303-player1.csv',
                    '2023-wimbledon-1303-player2.csv',
                    '2023-wimbledon-1304-player1.csv',
                    '2023-wimbledon-1304-player2.csv',
                    '2023-wimbledon-1305-player1.csv',
                    '2023-wimbledon-1305-player2.csv',
                    '2023-wimbledon-1306-player1.csv',
                    '2023-wimbledon-1306-player2.csv',
                    '2023-wimbledon-1307-player1.csv',
                    '2023-wimbledon-1307-player2.csv',
                    '2023-wimbledon-1308-player1.csv',
                    '2023-wimbledon-1308-player2.csv',
                    '2023-wimbledon-1309-player1.csv',
                    '2023-wimbledon-1309-player2.csv',
                    '2023-wimbledon-1310-player1.csv',
                    '2023-wimbledon-1310-player2.csv',
                    '2023-wimbledon-1311-player1.csv',
                    '2023-wimbledon-1311-player2.csv',
                    '2023-wimbledon-1312-player1.csv',
                    '2023-wimbledon-1312-player2.csv',
                    '2023-wimbledon-1313-player1.csv',
                    '2023-wimbledon-1313-player2.csv',
                    '2023-wimbledon-1314-player1.csv',
                    '2023-wimbledon-1314-player2.csv',
                    '2023-wimbledon-1315-player1.csv',
                    '2023-wimbledon-1315-player2.csv',
                    '2023-wimbledon-1316-player1.csv',
                    '2023-wimbledon-1316-player2.csv',
                    '2023-wimbledon-1401-player1.csv',
                    '2023-wimbledon-1401-player2.csv',
                    '2023-wimbledon-1402-player1.csv',
                    '2023-wimbledon-1402-player2.csv',
                    '2023-wimbledon-1403-player1.csv',
                    '2023-wimbledon-1403-player2.csv',
                    '2023-wimbledon-1404-player1.csv',
                    '2023-wimbledon-1404-player2.csv',
                    '2023-wimbledon-1405-player1.csv',
                    '2023-wimbledon-1405-player2.csv',
                    '2023-wimbledon-1406-player1.csv',
                    '2023-wimbledon-1406-player2.csv',
                    '2023-wimbledon-1407-player1.csv',
                    '2023-wimbledon-1407-player2.csv',
                    '2023-wimbledon-1408-player1.csv',
                    '2023-wimbledon-1408-player2.csv',
                    '2023-wimbledon-1501-player1.csv',
                    '2023-wimbledon-1501-player2.csv',
                    '2023-wimbledon-1502-player1.csv',
                    '2023-wimbledon-1502-player2.csv',
                    '2023-wimbledon-1503-player1.csv',
                    '2023-wimbledon-1503-player2.csv',
                    '2023-wimbledon-1504-player1.csv',
                    '2023-wimbledon-1504-player2.csv',
                    '2023-wimbledon-1601-player1.csv',
                    '2023-wimbledon-1601-player2.csv',
                    '2023-wimbledon-1602-player1.csv',
                    '2023-wimbledon-1602-player2.csv',
                    '2023-wimbledon-1701-player1.csv',
                    '2023-wimbledon-1701-player2.csv',
                    ]
    timestep = 10
    batch_size = 64
    feature_size = 12#16
    hidden_size = 60 # TODO: can change
    output_size = 4
    predict_len = 5
    num_layers = 2 # TODO: can change
    epochs = 600 # TODO: can change
    best_loss = 100
    learning_rate = 0.001 # TODO: can change
    model_name = 'GRU_for_tennis_momentum_swing'
    save_path = './trained_models/{}.pth'.format(model_name)
    loss_type = 2

config = Config()

In [111]:
import csv

def delete_columns(input_file, output_file, columns_to_delete):
    with open(input_file, 'r') as infile, open(output_file, 'w', newline='') as outfile:
        reader = csv.reader(infile)
        writer = csv.writer(outfile)
        
        for row in reader:
            # Exclude columns to delete (using 0-based index)
            filtered_row = [value for idx, value in enumerate(row) if idx not in columns_to_delete]
            writer.writerow(filtered_row)

# Example usage:
input_csv_file = config.input_paths[0]
output_csv_file = 'your_output_file.csv'
columns_to_delete = [10, 11, 14, 15]  # Columns are 0-based, so 11 becomes 10, 12 becomes 11, and so on.
for input_path in config.input_paths:
    delete_columns('./data_for_training/data/'+input_path, './data_for_training/data1/'+input_path, columns_to_delete)

In [122]:

def fetch_data(dataX, dataY, flag, timestep, feature_size, output_size, pred_len):
    myDataX = []
    myDataY = []
    for i in range(0, len(dataX) - timestep):
        target_row = dataY[i + timestep - 1].reshape(-1, output_size)
        res = []
        for j in range(pred_len):
            for k in range(output_size):
                if target_row[j][k] == 1:
                    if flag:
                        res.append(2 - k if k % 2 == 0 else 4 - k)
                    else:
                        res.append(k)
                    break
        if len(res) != pred_len:
            break
        myDataX.append(dataX[i: i + timestep])
        myDataY.append(res)
    myDataX = np.array(myDataX)
    myDataY = np.array(myDataY)

    train_size = int(np.round(0.8 * myDataX.shape[0]))
    trainX = myDataX[:train_size, :].reshape(-1, timestep, feature_size)
    testX = myDataX[train_size:, :].reshape(-1, timestep, feature_size)
    trainY = myDataY[:train_size, :].reshape(-1, pred_len)
    testY = myDataY[train_size:, :].reshape(-1, pred_len)
    return (trainX, trainY, testX, testY)

trainX = []
trainY = []
testX = []
testY = []
dfy = pd.read_csv('./data_for_training/momentum_condition.csv')
dataY = np.array(dfy)
df_idx = pd.read_csv('./data_for_training/match_index.csv')
match_id = np.array(df_idx).reshape(-1)
for i in range(len(match_id)):
    match_id[i] -= 1
pos = 0
match_no = 0

for input_path in config.input_paths:
    input_path = './data_for_training/data1/' + input_path
    dfx = pd.read_csv(input_path)
    dataX = np.array(dfx)
    a, b, c, d = fetch_data(dataX, dataY[pos: match_id[match_no // 2]], match_no % 2, config.timestep, config.feature_size, config.output_size, config.predict_len)
    if match_no % 2 == 1:
        pos = match_id[match_no // 2]
        print("match {:d}: point {:d}-{:d} in train set (for TWO times, one for each player), point {:d}-{:d} in test set (for TWO times, one for each player).".format(match_no // 2 + 1, config.timestep, config.timestep + len(a) - 1, config.timestep + len(a), config.timestep + len(a) + len(c) - 1))
    match_no += 1
    for x in a:
        trainX.append(x)
    for y in b:
        trainY.append(y)
    for x in c:
        testX.append(x)
    for y in d:
        testY.append(y)
trainX = np.array(trainX).reshape(-1, config.timestep, config.feature_size)
testX = np.array(testX).reshape(-1, config.timestep, config.feature_size)
trainY = np.array(trainY).reshape(-1, config.predict_len)
testY = np.array(testY).reshape(-1, config.predict_len)

x_train_tensor = torch.from_numpy(trainX).to(torch.float32)
y_train_tensor = torch.from_numpy(trainY).to(torch.long)

x_test_tensor = torch.from_numpy(testX).to(torch.float32)
y_test_tensor = torch.from_numpy(testY).to(torch.long)

train_data = TensorDataset(x_train_tensor, y_train_tensor)
test_data = TensorDataset(x_test_tensor, y_test_tensor)

train_loader = DataLoader(train_data, config.batch_size, False)
test_loader = DataLoader(test_data, config.batch_size, False)

match 1: point 10-237 in train set (for TWO times, one for each player), point 238-294 in test set (for TWO times, one for each player).
match 2: point 10-158 in train set (for TWO times, one for each player), point 159-195 in test set (for TWO times, one for each player).
match 3: point 10-104 in train set (for TWO times, one for each player), point 105-128 in test set (for TWO times, one for each player).
match 4: point 10-267 in train set (for TWO times, one for each player), point 268-331 in test set (for TWO times, one for each player).
match 5: point 10-194 in train set (for TWO times, one for each player), point 195-240 in test set (for TWO times, one for each player).
match 6: point 10-263 in train set (for TWO times, one for each player), point 264-326 in test set (for TWO times, one for each player).
match 7: point 10-183 in train set (for TWO times, one for each player), point 184-226 in test set (for TWO times, one for each player).
match 8: point 10-149 in train set (for T

In [123]:
# GRU
class GRURNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super().__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.gru = nn.GRU(self.input_size, self.hidden_size, self.num_layers, batch_first=True)
        self.fc1 = nn.Linear(self.hidden_size, output_size)
        self.fc2 = nn.Linear(self.hidden_size, output_size)
        self.fc3 = nn.Linear(self.hidden_size, output_size)
        self.fc4 = nn.Linear(self.hidden_size, output_size)
        self.fc5 = nn.Linear(self.hidden_size, output_size)
        self.softmax = nn.Softmax(dim=1)
 
    def forward(self, input_seq):
        batch_size = input_seq.shape[0]
        h_0 = torch.zeros(self.num_layers, batch_size, self.hidden_size)
        output, _ = self.gru(input_seq,h_0)
        pred1 = self.fc1(output)
        pred2 = self.fc2(output)
        pred3 = self.fc3(output)
        pred4 = self.fc4(output)
        pred5 = self.fc5(output)
        pred1, pred2, pred3, pred4, pred5 = pred1[:, -1, :], pred2[:, -1, :], pred3[:, -1, :], pred4[:, -1, :], pred5[:, -1, :]
        pred1, pred2, pred3, pred4, pred5 = self.softmax(pred1), self.softmax(pred2), self.softmax(pred3), self.softmax(pred4), self.softmax(pred5)
        pred = torch.stack([pred1, pred2, pred3, pred4, pred5], dim=1)
        return pred
    
model = GRURNN(config.feature_size, config.hidden_size, config.num_layers, config.output_size)
loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=config.learning_rate)

In [124]:
for epoch in range(config.epochs):
    model.train()
    running_loss = 0
    train_bar = tqdm(train_loader)
    for data in train_bar:
        x_train, y_train = data
        optimizer.zero_grad()
        y_train_pred = model(x_train)

        loss1 = loss_function(y_train_pred[:, 0, :], y_train[:, 0])
        loss2 = loss_function(y_train_pred[:, 1, :], y_train[:, 1])
        loss3 = loss_function(y_train_pred[:, 2, :], y_train[:, 2])
        loss4 = loss_function(y_train_pred[:, 3, :], y_train[:, 3])
        loss5 = loss_function(y_train_pred[:, 4, :], y_train[:, 4])
        
        if config.loss_type == 1:
            loss = max(loss1, loss2, loss3, loss4, loss5)
        if config.loss_type == 2:
            loss = loss1 + loss2 + loss3 + loss4 + loss5
        if config.loss_type == 3:
            loss = 5 * loss1 + 2 * loss2 + loss3 + loss4 + loss5
        
        loss.backward()
        optimizer.step()
        # print("loss =", loss)
        
        running_loss += loss.item()
        train_bar.desc = 'train epoch[{}/{}] loss: {:.3f}'.format(epoch + 1, config.epochs, loss)
    
    # model.eval()
    # test_loss = 0
    # with torch.no_grad():
    #     test_bar = tqdm(test_loader)
    #     for data in test_bar:
    #         x_test, y_test = data
    #         y_test_pred = model(x_test)
    #         loss1 = loss_function(y_test_pred[:, 0, :], y_test[:, 0])
    #         loss2 = loss_function(y_test_pred[:, 1, :], y_test[:, 1])
    #         loss3 = loss_function(y_test_pred[:, 2, :], y_test[:, 2])
    #         loss4 = loss_function(y_test_pred[:, 3, :], y_test[:, 3])
    #         loss5 = loss_function(y_test_pred[:, 4, :], y_test[:, 4])
    #         # test_loss = max(loss1, loss2, loss3, loss4, loss5)
    #         # test_loss = loss1 + loss2 + loss3 + loss4 + loss5
    #         test_loss = 5 * loss1 + 3 * loss2 + loss3 + loss4 + loss5
    
    # if test_loss < config.best_loss:
    #     config.best_loss = test_loss
    #     torch.save(model.state_dict(), config.save_path)


train epoch[277/600] loss: 4.388:   8%|▊         | 14/171 [00:00<00:01, 89.37it/s]


KeyboardInterrupt: 

In [125]:
# model = GRURNN(config.feature_size, config.hidden_size, config.num_layers, config.output_size)
# model.load_state_dict(torch.load(config.save_path))

model.eval()
with torch.no_grad():
    train_bar = tqdm(train_loader)

    predict_rate = [0 for _ in range(config.predict_len)]
    tot = [0 for _ in range(config.predict_len)]
    for data in train_bar:
        x_train, y_train = data
        y_train_pred = model(x_train)
        for i in range(config.predict_len):
            tot[i] += x_train.shape[0]
        for i in range(x_train.shape[0]):
            for j in range(config.predict_len):
                pred = 0
                for k in range(config.output_size):
                    if y_train_pred[i][j][k].item() > y_train_pred[i][j][pred].item():
                        pred = k
                predict_rate[j] += (1 if pred == y_train[i][j] else 0)
    for i in range(config.predict_len):
        print('train set predict rate for step {:d} is: {:f}'.format(i + 1, predict_rate[i] / tot[i]))
    test_bar = tqdm(test_loader)

    predict_rate = [0 for _ in range(config.predict_len)]
    tot = [0 for _ in range(config.predict_len)]
    for data in test_bar:
        x_test, y_test = data
        y_test_pred = model(x_test)
        for i in range(config.predict_len):
            tot[i] += x_test.shape[0]
        for i in range(x_test.shape[0]):
            for j in range(config.predict_len):
                pred = 0
                for k in range(config.output_size):
                    if y_test_pred[i][j][k].item() > y_test_pred[i][j][pred].item():
                        pred = k
                predict_rate[j] += (1 if pred == y_test[i][j] else 0)
    for i in range(config.predict_len):
        print('test set predict rate for step {:d} is: {:f}'.format(i + 1, predict_rate[i] / tot[i]))

100%|██████████| 171/171 [00:03<00:00, 48.30it/s]


train set predict rate for step 1 is: 0.794630
train set predict rate for step 2 is: 0.829729
train set predict rate for step 3 is: 0.839626
train set predict rate for step 4 is: 0.840359
train set predict rate for step 5 is: 0.804435


100%|██████████| 43/43 [00:00<00:00, 51.10it/s]

test set predict rate for step 1 is: 0.377109
test set predict rate for step 2 is: 0.342260
test set predict rate for step 3 is: 0.320249
test set predict rate for step 4 is: 0.307777
test set predict rate for step 5 is: 0.296405


In [126]:
# save model
if config.loss_type == 1:
    save_path = './trained_models/{}_max.pth'.format(config.model_name)
if config.loss_type == 2:
    save_path = './trained_models/{}_sum.pth'.format(config.model_name)
if config.loss_type == 3:
    save_path = './trained_models/{}_lc.pth'.format(config.model_name)
torch.save(model.state_dict(), save_path)

In [130]:
# load model and predict, then save the result
def compute_prediction(model, output_file):
    model.eval()
    
    lst = []
    column = ['is_train_set', 'pred_y1', 'pred_y2', 'pred_y3', 'pred_y4', 'pred_y5', 'target_y1', 'target_y2', 'target_y3', 'target_y4', 'target_y5']
    with torch.no_grad():
        train_bar = tqdm(train_loader)

        predict_rate = [0 for _ in range(config.predict_len)]
        tot = [0 for _ in range(config.predict_len)]
        for data in train_bar:
            x_train, y_train = data
            y_train_pred = model(x_train)
            for i in range(config.predict_len):
                tot[i] += x_train.shape[0]
            for i in range(x_train.shape[0]):
                row = [1]
                for j in range(config.predict_len):
                    pred = 0
                    for k in range(config.output_size):
                        if y_train_pred[i][j][k].item() > y_train_pred[i][j][pred].item():
                            pred = k
                    predict_rate[j] += (1 if pred == y_train[i][j] else 0)
                    row.append(pred)
                for j in range(config.predict_len):
                    row.append(y_train[i][j].item())
                lst.append(row)
        for i in range(config.predict_len):
            print('train set predict rate for step {:d} is: {:f}'.format(i + 1, predict_rate[i] / tot[i]))
        
        test_bar = tqdm(test_loader)

        predict_rate = [0 for _ in range(config.predict_len)]
        tot = [0 for _ in range(config.predict_len)]
        for data in test_bar:
            x_test, y_test = data
            y_test_pred = model(x_test)
            for i in range(config.predict_len):
                tot[i] += x_test.shape[0]
            for i in range(x_test.shape[0]):
                row = [0]
                for j in range(config.predict_len):
                    pred = 0
                    for k in range(config.output_size):
                        if y_test_pred[i][j][k].item() > y_test_pred[i][j][pred].item():
                            pred = k
                    predict_rate[j] += (1 if pred == y_test[i][j] else 0)
                    row.append(pred)
                for j in range(config.predict_len):
                    row.append(y_test[i][j].item())
                lst.append(row)
        for i in range(config.predict_len):
            print('test set predict rate for step {:d} is: {:f}'.format(i + 1, predict_rate[i] / tot[i]))
    df = pd.DataFrame(lst, columns = column)
    df.to_csv(output_file)

config.hidden_size = 60
model_max = GRURNN(config.feature_size, config.hidden_size, config.num_layers, config.output_size)
model_sum = GRURNN(config.feature_size, config.hidden_size, config.num_layers, config.output_size)
model_lc = GRURNN(config.feature_size, config.hidden_size, config.num_layers, config.output_size)
#model_max.load_state_dict(torch.load('./trained_models/GRU_for_tennis_momentum_swing_max.pth'))
model_sum.load_state_dict(torch.load('./trained_models/GRU_for_tennis_momentum_swing_sum_overfit.pth'))
#model_lc.load_state_dict(torch.load('./trained_models/GRU_for_tennis_momentum_swing_lc.pth'))
# print('-----------model_max------------')
# compute_prediction(model_max, './pred_res/model_max.csv')
print('-----------model_sum------------')
compute_prediction(model_sum, './pred_res/model_sum_overfit.csv')
# print('----model_linear_combination----')
# compute_prediction(model_lc, './pred_res/model_lc.csv')

-----------model_sum------------


  0%|          | 0/171 [00:00<?, ?it/s]

100%|██████████| 171/171 [00:05<00:00, 29.71it/s]


train set predict rate for step 1 is: 0.794630
train set predict rate for step 2 is: 0.829729
train set predict rate for step 3 is: 0.839626
train set predict rate for step 4 is: 0.840359
train set predict rate for step 5 is: 0.804435


100%|██████████| 43/43 [00:01<00:00, 35.00it/s]

test set predict rate for step 1 is: 0.377109
test set predict rate for step 2 is: 0.342260
test set predict rate for step 3 is: 0.320249
test set predict rate for step 4 is: 0.307777
test set predict rate for step 5 is: 0.296405


In [131]:
# feature importance
def calcPredictRate(model, test_loader):
    model.eval()
    predict_rate = [0 for _ in range(config.predict_len)]
    tot = [0 for _ in range(config.predict_len)]
    with torch.no_grad():
        
        test_bar = tqdm(test_loader)

        for data in test_bar:
            x_test, y_test = data
            y_test_pred = model(x_test)
            for i in range(config.predict_len):
                tot[i] += x_test.shape[0]
            for i in range(x_test.shape[0]):
                for j in range(config.predict_len):
                    pred = 0
                    for k in range(config.output_size):
                        if y_test_pred[i][j][k].item() > y_test_pred[i][j][pred].item():
                            pred = k
                    predict_rate[j] += (1 if pred == y_test[i][j] else 0)
    return np.array([predict_rate[i] / tot[i] for i in range(config.predict_len)])

def calcFeatureImportance(model):
    accuracy = calcPredictRate(model, test_loader)
    importance = np.zeros(config.feature_size)
    for i in range(config.feature_size):
        ntestX = testX.copy()
        nfeature = ntestX[:, :, i].copy().reshape(-1)
        np.random.shuffle(nfeature)
        nfeature = nfeature.reshape(-1, config.timestep)
        for j in range(ntestX.shape[0]):
            for k in range(ntestX.shape[1]):
                ntestX[j][k][i] = nfeature[j][k]
        nx_test_tensor = torch.from_numpy(ntestX).to(torch.float32)
        ntest_data = TensorDataset(nx_test_tensor, y_test_tensor)
        ntest_loader = DataLoader(ntest_data, config.batch_size, False)
        naccuracy = calcPredictRate(model, ntest_loader)
        importance[i] = sum([(accuracy[j] - naccuracy[j]) * (accuracy[j] - naccuracy[j]) for j in range(config.predict_len)])

    tot = np.sum(importance)
    for i in range(importance.shape[0]):
        importance[i] = importance[i] / tot
    
    return importance

column_name = ['model_name'] + ['f' + str(i + 1) for i in range(config.feature_size)]
z = [['model_sum'] + calcFeatureImportance(model_sum).tolist()]
#z = [['model_max'] + calcFeatureImportance(model_max).tolist(), ['model_sum'] + calcFeatureImportance(model_sum).tolist(), ['model_lc'] + calcFeatureImportance(model_lc).tolist()]

100%|██████████| 43/43 [00:00<00:00, 51.51it/s]


In [132]:
print(z)
importance = pd.DataFrame(data=z, columns=column_name)
importance.to_csv('./trained_models/feature_importance.csv')

[['model_sum', 0.021650982446062568, 0.004823531399415421, 0.03488823936880161, 0.4939600263562488, 0.02715031509233138, 0.07847741979083948, 0.07525046883711511, 0.012113737349845435, 0.11641521228606662, 0.03588504620791024, 0.05512003919647231, 0.04426498166889114]]
